## Timing Comparison for Training Loops

In [ ]:

import wandb
import config
import diamond_world_model_trainer as trainer

wandb.init(project='timing-comparison', reinit=True)


### Run `_main_training` on non-incremental dataset

In [ ]:

config.DATA_DIR = 'jetbot_data_two_actions_nonincremental_test'
config.NUM_EPOCHS = 1
trainer._main_training()
noninc_table = wandb.run.history[-1].get('train_epoch_perf')


### Run `train_diamond_model` on incremental dataset

In [ ]:

config.DATA_DIR = 'jetbot_data_two_actions_incremental_test'
train_loader = None  # TODO: provide DataLoader
val_loader = None
fresh_dataset_size = 200
trainer.train_diamond_model(train_loader, val_loader, fresh_dataset_size)
inc_table = wandb.run.history[-1].get('incremental_perf')


### Compare timings

In [ ]:

import pandas as pd

if noninc_table is not None and inc_table is not None:
    df_non = pd.DataFrame(noninc_table.data, columns=noninc_table.columns)
    df_inc = pd.DataFrame(inc_table.data, columns=inc_table.columns)
    display(df_non.describe())
    display(df_inc.describe())
